<a href="https://colab.research.google.com/github/vedantdave77/project.Orca/blob/master/Recurrent_Neural_Network/Sentiment-Analysis_using_RNN(LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis with RNN

##Intution:
Previously I worked on Sentiment analytics using simple machine learning and deep learning algorithm, but the Accuracy was too low. 

So, here I am going to develop the same kind of sentiment analysis using RNN. I will use PyTorch library. I am aspecting a higher accuracy for the network. 


## Load Data and Dependencies 


In [3]:
import numpy as np

with open('/content/reviews.txt','r') as f:
  reviews = f.read()
with open('/content/labels.txt', 'r') as f:
  label = f.read()

print(reviews[:1000])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

In [4]:
print(label[:100])

positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
n


## Data Preprocessing 

You can see an example of the reviews data above. Here are the processing steps, we'll want to take:

- We'll want to get rid of periods and extraneous punctuation.
- Also, you might notice that the reviews are delimited with newline characters \n. To deal with those,
-  I'm going to split the text into each review using \n as the delimiter.
- Then I can combined all the reviews back together into one big string.

In [14]:
from string import punctuation

# get rid of punctuation 
reviews = reviews.lower()
all_text = ''.join([c for c in reviews if c not in punctuation])
print(all_text[:30])

# split by line and space to make individual words 
reviews_split = all_text.split('\n')
print(reviews_split[1])
all_text = ' '.join(reviews_split)
print(all_text[:30])

# create list of words 
words = all_text.split(' ')
len(words)

bromwell high is a cartoon com
story of a man who has unnatural feelings for a pig  starts out with a opening scene that is a terrific example of absurd comedy  a formal orchestra audience is turned into an insane  violent mob by the crazy chantings of it  s singers  unfortunately it stays absurd the whole time with no general narrative eventually making it just too off putting  even those from the era should be turned off  the cryptic dialogue would make shakespeare seem easy to a third grader  on a technical level it  s better than you might think with some good cinematography by future great vilmos zsigmond  future stars sally kirkland and frederic forrest can be seen briefly   
bromwell high is a cartoon com


233454

In [35]:
words[0:30]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me']

## Encoding the words 

Teh best way is to create specific dictionary that map the words in the vocabulary to integer. So, We will have specific code (number) to represent each word. 

In [42]:
from collections  import Counter 

# Build a dictionary 
counts = Counter(words)
vocab = sorted(counts,key = counts.get, reverse = True)
vocab_to_integer = {word: i for i, word in enumerate(vocab,1)}

# same for review (1 and 0)
reviews_to_integer = []
for review in reviews_split:
  reviews_to_integer.append([vocab_to_integer[word] for word in review.split()])
                            

In [50]:
print('Unique words: ' , len((vocab_to_integer)))
print()

print('Tockenize the review: ',reviews_to_integer[:1])

Unique words:  74072

Tockenize the review:  [[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]]


## Encoding Labels

In [58]:
# converting in 0 and 1
labels_split = label.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])
print('total labels: ',len(encoded_labels))

total labels:  25001


In [64]:
review_length = Counter([len(x) for x in reviews_to_integer])
print('Zero length review: {}'. format(review_length[0]))
print('max length review: {}'.format(max(review_length)))

Zero length review: 1
max length review: 2514


Here, I need to remove such review which has empty revies. so first need to clear data.


In [66]:
print('Number of revies befoere removing outlier: ', len(reviews_to_integer))

# get index of reviews with length 0
non_zero_idx = [c for c, review in enumerate(reviews_to_integer) if len(review) != 0]

# now remove it 
reviews_int = [reviews_to_integer[i] for i in non_zero_idx]
encoded_labels = np.array([encoded_labels[i] for i in non_zero_idx])

print('Number of review after the cleanup: ',len(reviews_int))


Number of revies befoere removing outlier:  25001
Number of review after the cleanup:  25000


## Padding Sequences 

The length of the sequences are uneven so first we need to make it to some constant value, here I will took 200 words/review. For short review, I will add 0 in null values. 

Here, I will add zero to left. And, its called left padding. 





In [78]:
def pad_features(reviews_int,review_length):
  features = np.zeros((len(reviews_int),review_length),dtype=int)

  for i,row in enumerate(reviews_int):
    features[i,-len(row):] = np.array(row)[:review_length]
  return features

In [81]:
review_length = 200
features = pad_features((reviews_int), review_length=review_length)

assert len(features) == len(reviews_int) 
assert len(features[0]) == review_length 

print(features[:30,:10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34     3   162   748  2731     9   325]
 [    9    11 10171  5305  1946   689   444    22   280   673]
 [    0     0     0     0     0     0     0     0     0

## Data Set Preparation


In [90]:
split_fraction = 0.8

# split data into training, validation and test data

split_idx = int(len(features)*split_fraction)
train_x,remaining_x = features[:split_idx], features[split_idx:]
train_y,remaining_y = features[:split_idx], features[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

# vizulize data
print('\t\t\tFeatures Shapes ')
print('Training set: \t{}'.format(train_x.shape),
      '\nValidation set: {}'.format(val_x.shape),
      '\nTest Set:\t{}'.format(test_x.shape))



			Features Shapes 
Training set: 	(20000, 200) 
Validation set: (2500, 200) 
Test Set:	(2500, 200)


## DataLoader and Batching 

In [92]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# SHUFFLE training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[    1, 13792,     6,  ...,   939,    99,   257],
        [    0,     0,     0,  ...,    32,    31,  2175],
        [    0,     0,     0,  ...,   202,  1115,    18],
        ...,
        [    0,     0,     0,  ...,  5721,     9,   592],
        [   22,     1,   386,  ...,    37,   113,    15],
        [    0,     0,     0,  ...,     6,     3,  1400]])

Sample label size:  torch.Size([50, 200])
Sample label: 
 tensor([[    1, 13792,     6,  ...,   939,    99,   257],
        [    0,     0,     0,  ...,    32,    31,  2175],
        [    0,     0,     0,  ...,   202,  1115,    18],
        ...,
        [    0,     0,     0,  ...,  5721,     9,   592],
        [   22,     1,   386,  ...,    37,   113,    15],
        [    0,     0,     0,  ...,     6,     3,  1400]])


## Sentiment Network with PyTorch

The layers are as follows:

- An **embedding layer** that converts our word tokens (integers) into embeddings of a specific size.
- An **LSTM layer** defined by a hidden_state size and number of layers
- A **fully-connected output layer** that maps the LSTM layer outputs to a desired output_size
- A **sigmoid activation layer** which turns all outputs into a value 0-1; return only the last sigmoid output as the output of this network.

**The Embedding Layer** 

We need to add an embedding layer because there are 74000+ words in our vocabulary. It is massively inefficient to one-hot encode that many classes. So, instead of one-hot encoding, we can have an embedding layer and use that layer as a lookup table. You could train an embedding layer using Word2Vec, then load it here. But, it's fine to just make a new layer, using it for only dimensionality reduction, and let the network learn the weights.

**The LSTM Layer(s)**

We'll create an LSTM to use in our recurrent network, which takes in an input_size, a hidden_dim, a number of layers, a dropout probability (for dropout between multiple layers), and a batch_first parameter.

In [93]:
# first check gpu avaibility 
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [146]:
import torch.nn as nn 

class RNN_LSTM(nn.Module):
  def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob= 0.5):
    super(RNN_LSTM,self).__init__()
    self.output_size = output_size
    self.n_layers = n_layers
    self.hidden_dim = hidden_dim

    # embedding and LSTM layers
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout = drop_prob, batch_first = True)

    self.dropout = nn.Dropout(0.3)

    self.fc = nn.Linear(hidden_dim,output_size)
    self.sig = nn.Sigmoid()

  def forward(self,x,hidden):
    batch_size = x.size(0)

    # embedding and lstm output
    x = x.long()
    embeds = self.embedding(x)
    lstm_out, hidden = self.lstm(embeds, hidden)

    lstm_out = lstm_out.contiguous().view(-1,self.hidden_dim)

    out = self.dropout(lstm_out)
    out = self.fc(out)
    sig_out = self.sig(out)

    # reshape to be batch_size 
    sig_out = sig_out.view(batch_size, -1)
    sig_out = sig_out[:,-1]

    return sig_out, hidden

  def init_hidden(self, batch_size):
    weight = next(self.parameters()).data

    if (train_on_gpu):
      hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
           weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
    else:
      hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                    weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
      
      return hidden 

# run


In [157]:
import torch.nn as nn

class RNN_LSTM(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.4):
        """
        Initialize the model by setting up the layers.
        """
        super(RNN_LSTM, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [158]:
# set paramers and initialize network 

vocab_size = len(vocab_to_integer)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = RNN_LSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

RNN_LSTM(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.4)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


## Training 


In [159]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [160]:
# training parameters
epochs = 4
counter = 0
print_every = 100 
clip = 5  # gradient clipping for gradient explodation

if(train_on_gpu):
  net.cuda()

net.train()

for e in range(epochs):
  h = net.init_hidden(batch_size)

  # batch loop
  for inputs, labels in train_loader:
    counter +=1
    
    if(train_on_gpu):
      inputs, labels = inputs.cuda(), labels.cuda()

    h = tuple([each.data for each in h])

    # reset gradient 
    net.zero_grad()

    # get the output from the model 
    output,h = net(inputs, h)

    # calculate loss 
    loss = criterion(output.squeeze(), labels.float())
    loss.backward()

    nn.utils.clip_grad_norm_(net.parameters(),ckop)
    optimizer.step()

    # loss state
    if coutner % print_every == 0:
      val_h = net.init_hidden(batch_size)
      val_losses = []
      net.eval()
      for inputs, labels in valid_loader:
        val_h = tuple([each.data for each in val_h])
        if(train_on_gpu):
          inputs, labels = inputs.cuda(), labels.cuda()
        output, val_h = net(inputs, val_h)
        val_loss = criterion(output.squeeze(), labels.float())
        val_losses.append(val_loss.item())

      net.train()
      print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([50, 200])) that is different to the input size (torch.Size([50])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


ValueError: ignored